In [2]:
import vk

In [7]:
from IPython.display import clear_output

login=str(raw_input())
passw=str(raw_input())

clear_output()

session = vk.AuthSession('6082600', login, passw, scope="wall,messages")
vk_api = vk.API(session)


In [8]:
dialogs = vk_api.messages.getDialogs()

In [9]:
len(dialogs)

21

In [10]:
dialogs

[301,
 {u'body': u'\u0422\u043e\u0433\u0434\u0430 \u043d\u0430\u043f\u0438\u0448\u0443 \u0442\u0435\u0431\u0435 \u0433\u0434\u0435 \u0442\u043e \u0432 7',
  u'date': 1497966158,
  u'mid': 147307,
  u'out': 0,
  u'read_state': 1,
  u'title': u' ... ',
  u'uid': 1029514},
 {u'admin_id': 134651636,
  u'body': u'',
  u'chat_active': u'201202547,134651636,18401120,100774744,50286423,155226421,86065563,143447457,34527813,214125249,42185718,27056209,65182126,184216814,186649084,156081905,223595124,179602765,66149879,66653661',
  u'chat_id': 74,
  u'date': 1497964121,
  u'fwd_messages': [{u'body': u'59 \u0431\u0430\u043b\u043b\u043e\u0432 \u043b\u0438\u0442\u0440\u0430',
    u'date': 1497964106,
    u'uid': 134651636},
   {u'body': u'\u042f \u0440\u0430\u0434\u043e\u0441\u0442\u043d\u044b\u0439',
    u'date': 1497964109,
    u'uid': 134651636}],
  u'mid': 147300,
  u'out': 0,
  u'photo_100': u'https://pp.userapi.com/c638531/v638531813/42384/aUaJEkngbac.jpg',
  u'photo_200': u'https://pp.userap

In [11]:
for dialog in dialogs:
    if type(dialog) == int:
        continue
    if u'body' in dialog:
        if dialog[u'body'] == u'Кайф)))))':
            print dialog[u'uid']

53427321


In [12]:
target_id = 53427321

In [13]:
from time import sleep
from tqdm import tqdm_notebook

In [14]:
class Message:
    def __init__(self,from_id, msg, replys=[]):
        self.from_id= from_id
        self.message= msg
        self.replys = replys

In [15]:

MAX_COUNT= 200
total_messages = vk_api.messages.getHistory(user_id = target_id, count=MAX_COUNT)[0]

In [17]:
messages = []


for i in tqdm_notebook(xrange(total_messages / MAX_COUNT+1)):
    sleep(0.3)
    tmp_messages = vk_api.messages.getHistory(user_id = target_id,offset=i*MAX_COUNT, count=MAX_COUNT)[1:]
    for msg in tmp_messages:
        txt = msg[u'body']
        txt = txt if len(txt)>0 else 'attachment'
        messages.append(Message(msg[u'from_id'],txt))

messages = list(reversed(messages))

In [18]:
import numpy as np

In [19]:
ids_seq = [msg.from_id for msg in messages]

In [20]:
len(ids_seq)

14495

In [31]:
changes = np.pad(np.where(np.diff(ids_seq) != 0)[0]+1,(1,1),'constant')
changes[-1] = len(ids_seq)

In [32]:
from collections import defaultdict

In [81]:
user_replies = defaultdict(lambda : defaultdict(set))

def merge_range(from_ind, to_ind):
    return '\n'.join([msg.message for msg in messages[from_ind:to_ind]])
        

    

In [82]:


for i in xrange(changes.shape[0] - 2):
    reply = merge_range(changes[i+1],changes[i+2]).lower()
    for j in xrange(changes[i], changes[i+1]):
        user_replies[messages[changes[i+1]].from_id][messages[j].message.lower()].add(reply)

In [83]:
from random import choice

In [138]:
from difflib import get_close_matches

In [230]:
def flatten(l):
    res = []
    for i in l:
        res += i
    return res

In [408]:

def get_answer(ind,txt):
    all_ent_quests = user_replies[ind].keys()

    msgs = get_close_matches(txt.lower(), all_ent_quests,n=5, cutoff=0)

    res = flatten([list(user_replies[ind][msg]) for msg in msgs])
    return choice(res)

In [412]:
start = u'Привет'

for i in xrange(10):
    print 'ENT>',start #FIXME
    ans = get_answer(53427321,start)
    print ''
    print 'Pidr>',ans
    start = get_answer(229991373,ans)
    print
print start

ENT> Привет

Pidr> брид ит бэр ит <br>мэйк ит йо нааакатик<br>бегин фэйс даун он зэ фло<br>брид ит бэр ит <br>мэйк ит йо нааакатик<br>соу йо нау зей вонт мооо

ENT> ебать
нихуя ты настрочил
а я вчера всю ночь на скорой катался

Pidr> пиздобол
ты дрочить ушел
а потом спать
я пойду

ENT> неа

Pidr> эт наш отряд
моц

ENT> лул
потом посмотрю
http://www.youtube.com/playlist?list=llrmnnq3v4xsg0jdry4fya5a
блть
attachment
группа - dethklok
у них есть свой мультик на 2х2
а сама группа состоит из 2 чуваков
для озвучки мультика пригласили участников популярных групп
из метеллицы, canobal corpse
и другие
вооот

Pidr> я видел

ENT> чув
а нахуя нужны шляпки и мячики?

Pidr> шляпки?

ENT> спок

Pidr> я решил проблему с 8 марта
http://vk.com/wall-45595714_5336533

ENT> лалз
зайди за коалку
attachment
хули ты не спишь?

Pidr> ок

ENT> рассказывай

Pidr> та ж хуйня)

ENT> окм
окс

Pidr> го
я выхожу
точнее
выбегаю
за мной гонится мамка с топором

ENT> ага
лось
муломь
блять

Pidr> не

и когда будешь здоро

In [28]:
print merge_range(0,100)

Йоу, бро.
attachment
Пхаха
Я еще с кошакером общался
и телку выебал
Ахаха, разведал
?
?
Что он задумал?
Кто?
Кошакер
Пингвинский?
Аааа
не
он хороший
Чувак, позже реально
оки
Короче нихуя не получилось
Пачиму?
У меня запросили номер моб тел, я ввел, пришло смс ебать ты лох
Ахаха
ахаха
Нет
Пришел код, я его сука ввел, мне написали, этот номер уже прифязан к другой странице, типа нельзя создать (можно если удалишь страницу Павел Смирнов)
так и что?ъ
удалил бы
-_-
Нахуя?
ебать нахуя?
ты можешь быть коалой!!!
а не человеком
ты же всегда мечтал!
Ахаха, кошка мурка))
Кошакер
моя подруга)
Мне нужна и та и другая страница
зачем?
я пошутил
кароч
пздс
ну ок
купи себе новую сим ку
Не, номер брата-пиздюка напишу
вооот
или так
А потом отвяжу номер
Пох, завтра сделаю
Как мило
attachment
Дорогой Енот
ахаахах
Пхаха, "о себе"
Побег енота
😆
ну что го дружится Коала
Чувааааак
Я в туалете
Ну если тпк срочно
аахах
То
Привет
Ты
не
Коала
Почъэму
хуесос-человек, не смей меня обманывать
Я же енот, а не долбоеб


In [37]:
for msg in reversed(messages):
    print "ENT" if msg[u'from_id'] != target_id else "PIDR",">",msg[u'body']

ENT > Я тебя понял, ты дачник
PIDR > Ну давай я узнаю, как домой приду
PIDR > Не-не
PIDR > Я в Московии
ENT > Ааа
ENT > Тогда ок
ENT > Напиши, через сколько примерно будет дома
ENT > Успею я в спортзал сходить или нет
PIDR > Я думаю, успеешь
ENT > В 4 примерно тогда?
PIDR > Я еще ничего не узнал
PIDR > До дома не дошёл
ENT > Тогда попутного ветра, епты
ENT > Насколько там зубодробительный мороз на улице?
PIDR > Пасиба бля
PIDR > Ну такой
PIDR > Терпимо
ENT > Ок
PIDR > Так
PIDR > Мне надо будет расписаться в шкалке
PIDR > Заодно может и встретимся
PIDR > На часик
PIDR > Как такое?
ENT > Я все равно в спортзале пока
PIDR > Во сколько свободен будешь?
ENT > Вернусь к 4, удобно?
ENT > Могу и раньше, у меня безлимит
PIDR > Ну нормально, пока тогда с фотографиями разберусь
ENT > Ок, потому что если тебе удобно сейчас, то го сначала погуляем
PIDR > Если что, то звякну тогда. Номер старый же?
ENT > 89167155347
ENT > Сейчас неудобно?
PIDR > Падажжи ебана
PIDR > Ща
PIDR > Секунду
PIDR > Давай се